In [1]:
import tensorflow as tf_load_data
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = tf_load_data.keras.datasets.mnist.load_data()

In [2]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# 예제용 reshape
x_train, x_test = x_train/255.0, x_test/255.0   #Feature scaling 적용, 그대록 학습시 문제됨...

# [[], [], ...] --> [...]
x_train_rs = x_train.reshape(x_train.shape[0], 784).astype('float32')
x_test_rs = x_test.reshape(x_test.shape[0], 784).astype('float32')

# one hot encoding
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)

In [4]:
print(x_train_rs.shape)
print(y_train_onehot.shape)
print(y_test_onehot.shape)
print(x_test_rs.shape)

(60000, 784)
(60000, 10)
(10000, 10)
(10000, 784)


In [5]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import matplotlib.pyplot as plt
import random

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
tf.set_random_seed(777)

In [7]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [8]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [9]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white)
#[n개, 가로, 세로, color]
Y = tf.placeholder(tf.float32, [None, 10])

# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01)) # Filter : 3x3, 32개 / Color=1
#    Conv     -> (?, 28, 28, 32)
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME') # Stride = 1
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], # Pooling : 2x2
                    strides=[1, 2, 2, 1], padding='SAME') 
'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
'''

# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01)) # 32 : 직전 Laryer의 Filter 수와 동일
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1],
                    strides=[1, 2, 2, 1], padding='SAME')
L2_flat = tf.reshape(L2, [-1, 7 * 7 * 64]) # [7 * 7 * 64개 값], [7 * 7 * 64개 값], ... n개
'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0", shape=(?, 3136), dtype=float32)
'''

# Final FC 7x7x64 inputs -> 10 outputs
W3 = tf.get_variable("W3", shape=[7 * 7 * 64, 10],
                     #initializer=tf.contrib.layers.xavier_initializer())
                     initializer = tf.truncated_normal_initializer(stddev=0.1))
b = tf.Variable(tf.random_normal([10]))
logits = tf.matmul(L2_flat, W3) + b

In [10]:
# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [11]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    #total_batch = int(mnist.train.num_examples / batch_size)
    total_batch = 100

    for i in range(total_batch):
        #batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs, batch_ys = next_batch(batch_size, x_train_rs, y_train_onehot)
        feed_dict = {X: batch_xs, Y: batch_ys}
        
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      #X: mnist.test.images, Y: mnist.test.labels}))
      X: x_test_rs, Y: y_test_onehot}))

# Get one and predict
#r = random.randint(0, mnist.test.num_examples - 1)
r = random.randint(0, x_test.shape[0] - 1)
#print("Label: ", sess.run(tf.argmax(mnist.test.labels[r:r + 1], 1)))
print("Label: ", sess.run(tf.argmax(y_test_onehot[r : r + 1], 1)))
#print("Prediction: ", sess.run(
#    tf.argmax(logits, 1), feed_dict={X: mnist.test.images[r:r + 1]}))
print(
        "Prediction: ",
        sess.run(tf.argmax(logits, 1), feed_dict={X: x_test_rs[r : r + 1]})
)

Learning started. It takes sometime.
Epoch: 0001 cost = 1.164632556
Epoch: 0002 cost = 0.281371681
Epoch: 0003 cost = 0.204629647
Epoch: 0004 cost = 0.152719526
Epoch: 0005 cost = 0.136434218
Epoch: 0006 cost = 0.116779831
Epoch: 0007 cost = 0.106964453
Epoch: 0008 cost = 0.098897272
Epoch: 0009 cost = 0.100141171
Epoch: 0010 cost = 0.085301948
Epoch: 0011 cost = 0.080171770
Epoch: 0012 cost = 0.075726259
Epoch: 0013 cost = 0.067431328
Epoch: 0014 cost = 0.072172468
Epoch: 0015 cost = 0.065218946
Learning Finished!
Accuracy: 0.9806
Label:  [2]
Prediction:  [2]
